# InSituPy demonstration
Example dataset: https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast

In [22]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from insitupy import XeniumData
from pathlib import Path

## Load data

In [6]:
# input directories
data_dirs = [
    Path("C:/Users/ge37voy/data/2301_CRC/2301-01_20230504/01_xenium_data/output-XETG00050__0003621__Region_1__20230504__121954"),
    Path("C:/Users/ge37voy/data/datasets/Xenium_FFPE_Human_Breast_Cancer_Rep1_outs/output-XETG00000__slide_id__sample_id/"),
    Path("C:/Users/ge37voy/data/20230811__134602__2314_OTCs_Wollenberg/output-XETG00050__0003555__OTC0304-1__20230811__134819")
]

In [7]:
img_dirs = [elem.parent / "unregistered_images" for elem in data_dirs]

In [8]:
i = 0
data_dir = data_dirs[i]
img_dir = img_dirs[i]

In [9]:
data_dir

WindowsPath('C:/Users/ge37voy/data/2301_CRC/2301-01_20230504/01_xenium_data/output-XETG00050__0003621__Region_1__20230504__121954')

In [10]:
xd = XeniumData(data_dir)

In [11]:
xd

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium

## Read different data modalities

In [12]:
xd.read_annotations()
xd.read_images()
xd.read_matrix()

### Show overview of data

In [13]:
xd

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium
    ➤ images
       nuclei:	(27352, 34139)
       HE:	(27352, 34139, 3)
       panCK:	(27352, 34139, 3)
    ➤ matrix
       AnnData object with n_obs × n_vars = 151305 × 280
	       obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
	       var: 'gene_ids', 'feature_types', 'genome'
	       obsm: 'spatial'
    ➤ annotations
       tanja:	41 annotations, 5 classes ('Tumor', 'Fat', 'Mucus', 'Immune cells', 'Peritumoral Zone') 

## View data interactively

In [36]:
xd.show()

Viewer(camera=Camera(center=(0.0, 2906.04375, 3627.1625), zoom=0.05578577545432772, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 5812.299999999999, 0.2125), (0.0, 7254.537499999999, 0.2125)), current_step=(13675, 17069), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x201afcc8f40>, <Image layer 'HE' at 0x201716a0a00>, <Image layer 'panCK' at 0x201afbea100>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000201025DB940>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap

### Crop data

In [37]:
xx = xd.crop(shape_layer="Shapes", inplace=False)

napari.Viewer: napari


c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [23]:
xx.show()

Viewer(camera=Camera(center=(0.0, 368.90000000000003, 550.90625), zoom=0.2543045756675211, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 738.0125, 0.2125), (0.0, 1102.025, 0.2125)), current_step=(1736, 2592), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x20178d4bdc0>, <Image layer 'HE' at 0x2017d0d6760>, <Image layer 'panCK' at 0x2017e0cf130>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000201025DB940>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

## Perform preprocessing steps

In [38]:
xx.normalize(transformation_method="sqrt")
xx.hvg()
xx.reduce_dimensions(umap=True, tsne=False)

Store raw counts in anndata.layers['counts']...
Normalization, log-transformation...
Calculate highly-variable genes across all samples using seurat flavor...
Dimensionality reduction...


c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\scanpy\preprocessing\_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))
c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Leiden clustering...


In [26]:
xx.matrix.X

<6876x280 sparse matrix of type '<class 'numpy.float32'>'
	with 330540 stored elements in Compressed Sparse Row format>

In [21]:
xx.matrix.obsm['X_umap']

array([[ 6.31623   , 11.972955  ],
       [ 6.3744297 , 12.134667  ],
       [ 6.5023217 , 13.059262  ],
       ...,
       [-0.18682556, 19.206476  ],
       [ 0.9949526 , 16.405897  ],
       [-0.59261817, 16.641123  ]], dtype=float32)

In [71]:
xx.show()

Viewer(camera=Camera(center=(0.0, 344.0375, 481.20625), zoom=1.2128338525422735, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 688.2875, 0.2125), (0.0, 962.625, 0.2125)), current_step=(1619, 2264), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x1a612167910>, <Image layer 'HE' at 0x1a612269940>, <Image layer 'panCK' at 0x1a61e6c3520>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001A48CCD4A60>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [57]:
xx.show(keys=["leiden", "ACTA2", "LYZ"])

Viewer(camera=Camera(center=(0.0, 344.0375, 481.20625), zoom=0.4608752110115569, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 688.4727430774632, 0.2125), (0.0, 962.7127100054628, 0.2125)), current_step=(1619, 2264), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x1a5f35fa910>, <Image layer 'HE' at 0x1a5ee1f4670>, <Image layer 'panCK' at 0x1a5f5863df0>, <Points layer 'leiden' at 0x1a5f7003700>, <Points layer 'ACTA2' at 0x1a5edb86d30>, <Points layer 'LYZ' at 0x1a5f72138e0>], help='use <5> for transform, use <2> for add points, use <3> for select points', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_dou

In [72]:
screenshot = xx.viewer.screenshot()

## Save results

In [19]:
out_dir = data_dir.parent / "cropped_processed"

In [21]:
xx.save(out_dir, overwrite=True)

In [39]:
brain = XeniumData(r"C:\Users\ge37voy\data\20231031__144605__2303-01_Korn_Brain\output-XETG00050__0004111__CNS4__20231031__144734")

In [40]:
brain

XeniumData
Slide ID:	0004111
Region ID:	CNS4
Data path:	C:\Users\ge37voy\data\20231031__144605__2303-01_Korn_Brain
Data folder:	output-XETG00050__0004111__CNS4__20231031__144734
Metadata file:	experiment.xenium

In [41]:
brain.read_matrix()
brain.read_images()

In [42]:
brain.show()

Viewer(camera=Camera(center=(0.0, 3266.3375, 4529.96875), zoom=0.0532662262766069, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 6532.8875, 0.2125), (0.0, 9060.15, 0.2125)), current_step=(15371, 21317), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x201895b0e50>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x00000201025DB940>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [48]:
import numpy as np